In [2]:
import numpy as np 

In [3]:
A=np.random.rand(3,5)

In [4]:
A

array([[0.65374858, 0.83374671, 0.42983523, 0.70842116, 0.74123305],
       [0.27890041, 0.66909147, 0.59856629, 0.97145692, 0.69781088],
       [0.27750559, 0.34875061, 0.91043214, 0.56799442, 0.11846994]])

In [5]:
np.linalg.svd(A)

(array([[-0.63337978,  0.51753808,  0.57531243],
        [-0.63921839,  0.06911802, -0.76591289],
        [-0.43615354, -0.85286402,  0.2870419 ]]),
 array([2.34818439, 0.64957143, 0.31819483]),
 array([[-0.30380257, -0.47180396, -0.44798515, -0.56103156, -0.41189553],
        [ 0.18618784,  0.27757548, -0.78920805, -0.07796201,  0.50927253],
        [ 0.76101802,  0.21152168,  0.15767726, -0.54510525, -0.23261194],
        [ 0.54187856, -0.67039322, -0.19139834,  0.46576933, -0.05801859],
        [-0.01604024, -0.45407531,  0.33906417, -0.40630619,  0.71659548]]))

In [10]:
import scipy
import scipy.linalg

In [22]:
from datetime import datetime

In [6]:
M=np.random.rand(3,3)


In [7]:
M

array([[0.40365031, 0.4762823 , 0.02210425],
       [0.34702748, 0.07474289, 0.75307898],
       [0.48926788, 0.76680419, 0.2413447 ]])

In [69]:
P,L,U=scipy.linalg.lu(M)

In [158]:
U

array([[ 0.48926788,  0.76680419,  0.2413447 ],
       [ 0.        , -0.46913528,  0.58189823],
       [ 0.        ,  0.        , -0.37092305]])

In [14]:
L

array([[1.        , 0.        , 0.        ],
       [0.70927909, 1.        , 0.        ],
       [0.82500879, 0.33324693, 1.        ]])

## backward & forward substitution

In [147]:
def timer(func):
    def layer(*args,**kwargs):
        #print(args)
        def count(*args,**kwargs):
            start=datetime.now()
            res=func(*args,**kwargs)
            print(datetime.now()-start)
            return res
        return count(*args,**kwargs)
    return layer

In [152]:
@timer
def forward(L,b):
    d=np.shape(L)[0]
    Y=np.zeros(d)
    for i in range(d):
        l=L[i]
        Y[i]=b[i]
        for c in range(i):
            Y[i]-=Y[c]*l[c]
    return Y

In [171]:
@timer
def backward(U,b):
    d=np.shape(U)[0]
    Y=np.zeros(d)
    for i in range(d-1,-1,-1):
        #i=d-m-1
        l=U[i]
        Y[i]=b[i]
        for c in range(d-1,i,-1):
            Y[i]-=Y[c]*l[c]
        Y[i]=Y[i]/l[i]
    return Y

In [172]:
b=np.random.rand(np.shape(U)[0])

In [173]:
Y=backward(U,b)
print(Y)

0:00:00
[ 8.45918168 -4.15556316 -2.42622131]


In [174]:
np.dot(np.linalg.inv(U),b)

array([ 8.45918168, -4.15556316, -2.42622131])

In [175]:
forward(L,b)

0:00:00


array([0.36674702, 0.27758142, 0.50486873])

In [176]:
np.dot(np.linalg.inv(L),b)

array([0.36674702, 0.27758142, 0.50486873])

## time consuming

In [64]:
d=100
L=np.random.rand(d,d)
b=np.random.rand(d)
forward(L,b)

0:00:00.013003


In [ ]:
@timer()

In [66]:
@timer
def inv(L,b):
    res=np.dot(np.linalg.inv(L),b)

In [67]:
inv(L,b)

0:00:00.041887


In [47]:
b=np.array([1,2,3])

In [21]:
np.dot(np.linalg.inv(L),b)

array([1.        , 1.29072091, 1.74486243])